# [CSCI 3397/PSYC 3317] Lab 3: More about Filtering

**Posted:** Monday, February 2, 2026

**Due:** Monday, February 9, 2026

__Total Points__: 10 pts

__Submission__: please rename the .ipynb file as __\<your_username\>_lab3.ipynb__ before you submit it to canvas. Example: weidf_lab3.ipynb.

In [ ]:
# install new packages
! pip install opencv-python

In [ ]:
from imageio import imread, imwrite
import numpy as np
import matplotlib.pyplot as plt


# download image
! wget https://bc-cv.github.io/csci3397/public/dip_feature/cell_canny_im.png -O cell_canny_im.png
! wget https://bc-cv.github.io/csci3397/public/bc_eagle.png -O bc_eagle.png
! wget https://bc-cv.github.io/csci3397/public/dip_pixel/xray_image.png -O xray_image.png

! wget https://bc-cv.github.io/csci3397/public/dip_feature/einstein.png -O einstein.png
! wget https://bc-cv.github.io/csci3397/public/dip_preprocess/brightfield_bacteria.jpg -O brightfield_bacteria.jpg

# <b>0. OpenCV package</b>

## 0.1 Image Basics
Things are the same for grayscale images. <b>BUT FOR COLOR IMAGES</b>, opencv reads in the BGR order instead of RGB... Need to define `cv2_imshow` to display the image properly. Thus, in this course, we'll use `imageio.imread` instead.

In [ ]:
import cv2
import matplotlib.pyplot as plt
# read in BGR instead of RGB
I = cv2.imread('bc_eagle.png')
cv2_imshow(I)


In [ ]:
def cv2_imshow(I):
    if I.ndim == 3:
        I = cv2.cvtColor(I, cv2.COLOR_BGR2RGB)
        plt.imshow(I)
    else:
        plt.imshow(I, cmap='gray')

## 0.2 Pixel-level processing

We can directly use the histogram equalization function.
https://docs.opencv.org/4.x/d5/daf/tutorial_py_histogram_equalization.html

In [ ]:
img = cv2.imread('xray_image.png',0)
equ = cv2.equalizeHist(img)
res = np.hstack((img,equ)) #stacking images side-by-side
cv2_imshow(res)

## 0.3 Patch-level processing
As you now have the hand-on knowledge of convolution, we will directly use OpenCV functions to implement filtering. From now on, you can focus on learning how to use these filters to solve real-world problems.

Syntax:
- Generic filter: `cv2.filter2D(image, -1, kernel matrix)`
- Gaussian: `cv2.GaussianBlur(image, kernel_size, sigma_x, sigma_y)` (if sigma_x=0, it'll be automatically estimated)

[[Tutorial]](https://docs.opencv.org/4.x/d4/d13/tutorial_py_filtering.html)

In [ ]:

kernel_size = 21
sigma = 5

# Box filter
# note the grid ghosting artifacts
I_box = cv2.filter2D(I, -1, np.ones([kernel_size,kernel_size])/(kernel_size**2))

# Gaussian filter
I_gauss = cv2.GaussianBlur(I, (kernel_size,kernel_size), sigma)

# motion blur filter
# 1D version of box filter, as if the object moves really fast in one direction
I_hblur = cv2.filter2D(I, -1, np.ones([1,kernel_size])/kernel_size)
I_vblur = cv2.filter2D(I, -1, np.ones([kernel_size,1])/kernel_size)


print('box vs. Gaussian filter')
cv2_imshow(np.hstack([I_box,I_gauss]))

print('vertical vs. horizontal motion blur')
cv2_imshow(np.hstack([I_vblur,I_hblur]))

# <b> More Filtering</b>

## 1.1 Edge Filter
Lec. 7, page 16

In [ ]:
# direct difference filter: [1, -1]
kernel_diff = np.array([1,-1])

# compute the edge on the 0-th channel
Iy = cv2.filter2D(I[:,:,0], -1, kernel_diff)
Ix = cv2.filter2D(I[:,:,0], -1, kernel_diff.reshape([1,2]))
cv2_imshow(Ix)
cv2_imshow(Iy)

## 1.2 Gaussian Filter

### 1D Gaussian: mean and standard deviation
 Lec 8, page 7

In [ ]:
from matplotlib import pyplot as plt
import numpy as np


def gaussian(x, mu, sig):
    return (
        1.0 / (np.sqrt(2.0 * np.pi) * sig) * np.exp(-np.power((x - mu) / sig, 2.0) / 2)
    )


x_values = np.linspace(-3, 3, 100)
plt.subplot(211)
mu=0
for sig in [1,2,3]:
    plt.plot(x_values, gaussian(x_values, mu, sig))
plt.legend(['sig=1','sig=2','sig=3'])
plt.title('mu=0, different sigma')

plt.subplot(212)
sig=1
for mu in [-2, 0, 2]:
    plt.plot(x_values, gaussian(x_values, mu, sig))
plt.legend(['mu=-2','mu=0','mu=2'])
plt.title('sigma=1, different mu')
plt.show()

### Recursive nature
 Lec 8, page 10

In [ ]:
I = cv2.imread('bc_eagle.png')

kernel_size = 11
sigma = 2
I_gauss_1 = cv2.GaussianBlur(I, (kernel_size,kernel_size), sigma)
I_gauss_2 = cv2.GaussianBlur(I_gauss_1, (kernel_size,kernel_size), sigma)
I_gauss_sqrt2 = cv2.GaussianBlur(I, (kernel_size,kernel_size), sigma * np.sqrt(2))


cv2_imshow(I_gauss_1)
cv2_imshow(I_gauss_2)
cv2_imshow(I_gauss_sqrt2)

### Separable nature
 Lec 8, page 11

In [ ]:
kernel_size = 21
sigma = 110

I_gauss_1 = cv2.GaussianBlur(I, (kernel_size,kernel_size), sigma)

I_gauss_h = cv2.GaussianBlur(I, (kernel_size,1), sigma)
I_gauss_v = cv2.GaussianBlur(I, (1,kernel_size), sigma)

I_gauss_hv = cv2.GaussianBlur(I_gauss_h, (kernel_size,1), sigma)

cv2_imshow(I_gauss_1)
cv2_imshow(I_gauss_h)
cv2_imshow(I_gauss_v)
cv2_imshow(I_gauss_hv)

## 1.3 Gaussian Derivative Filter

Convolve with Gaussian derivative (GD) filter.

(Lec. 8, slide 14)


In [ ]:
# load input image
I_cell = imread('cell_canny_im.png')
cv2_imshow(I_cell)
plt.axis('off')
plt.title('input image');

In [ ]:
from imageio import imread
import matplotlib.pyplot as plt
import cv2

I_cell2 = I_cell[::2,::2]
I_cell2 = np.clip(I_cell2.astype(float)  +  20 * np.random.random(I_cell2.shape),0,255)

# direct difference filter: [1, -1]
kernel_diff = np.array([1,-1])
Iy = cv2.filter2D(I_cell2, -1, kernel_diff)
Ix = cv2.filter2D(I_cell2, -1, kernel_diff.reshape([1,2]))

# GD filter
kernel_size = 5
I_guass = cv2.GaussianBlur(I_cell2, [kernel_size,kernel_size], 0)
Iy_GD = cv2.filter2D(I_guass, -1, kernel_diff)
Ix_GD = cv2.filter2D(I_guass, -1, kernel_diff.reshape([1,2]))

# show the image
plt.figure(figsize=(8, 8))
# show the pixel value histogram
plt.subplot(131)
cv2_imshow(Iy)
plt.axis('off')
plt.title('Dy filter')

plt.subplot(132)
cv2_imshow(np.abs(Iy) > 10)
plt.axis('off')
plt.title('Dy filter+threshold: noisy')

plt.subplot(133)
cv2_imshow(np.abs(Iy_GD)>10)
plt.axis('off')
plt.title('GD-y filter+threshold: less noisy')


plt.show()

# [10 pts] Exercise

## (1) [2 pts] Sharpen filter with paramter $\alpha$

Implement the sharpen filter with OpenCV. i.e., only need to code up the filter kernel and use `cv2.filter2D`. The sharpen kernel in the lecture note sets $\alpha=1$.

Lec 7, page 5

In [ ]:
im_xray = imread('xray_image.png')

kernel_size = 3


### Your code starts here
im_xray_sharpen_alpha_1 = ???
im_xray_sharpen_alpha_2 = ???
### Your code ends here

plt.figure(figsize=(18, 8))

plt.subplot(221)
cv2_imshow(im_xray)
plt.title('original image')
plt.subplot(222)
cv2_imshow(im_xray_sharpen_alpha_1)
plt.title('alpha=1')
plt.subplot(223)
cv2_imshow(im_xray_sharpen_alpha_2)
plt.title('alpha=2')

In [ ]:
im_xray.max()

## (2) [3 pts] Canny Edge

Find an online opencv tutorial on how to apply canny edge detection and generate the result.


In [ ]:
im_cell = imread('cell_canny_im.png')

### Your code starts here
im_cell_canny =
### Your code ends here

plt.figure(figsize=(18, 8))

plt.subplot(121)
cv2_imshow(im_cell)
plt.title('original image')
plt.subplot(122)
cv2_imshow(im_cell_canny)
plt.title('canny edge detection results')

## (3) [3 pts] Bilateral Filtering

Use **OpenCV's `cv2.bilateralFilter`** on the Einstein image. Bilateral filtering smooths the image while preserving edges by combining a spatial kernel with a range (intensity) kernel. 

Fill in the parameters below and compare with Gaussian blur to see edge preservation.
(lec. 8, page 26-27)

In [ ]:
from imageio import imread
import cv2
import matplotlib.pyplot as plt

# Load Einstein image (grayscale for display)
I_einstein = imread('einstein.png')

# Bilateral filter parameters (fill in)
# d: diameter of pixel neighborhood
# sigma_color: filter sigma in the color/intensity space
# sigma_space: filter sigma in the coordinate space
d = ???
sigma_color = ???
sigma_space = ???

I_einstein_bilateral = cv2.bilateralFilter(I_einstein, d, sigma_color, sigma_space)

# Compare with Gaussian blur (same spatial extent, no edge preservation)
I_einstein_gaussian = cv2.GaussianBlur(I_einstein, (???, ???), ???)

plt.figure(figsize=(12, 4))
plt.subplot(131)
cv2_imshow(I_einstein)
plt.title('Original')
plt.axis('off')
plt.subplot(132)
cv2_imshow(I_einstein_gaussian)
plt.title('Gaussian blur')
plt.axis('off')
plt.subplot(133)
cv2_imshow(I_einstein_bilateral)
plt.title('Bilateral filter')
plt.axis('off')
plt.tight_layout()
plt.show()

## (4) [2 pts] Vibe coding GUI for Bilateral Filtering

Use your AI coding IDE to generate a ipython widget that
- load the 'einstein.png' image
- have a sliding bar for each of the three bilateral filtering parameters, so that you can drag the sliding bar to see the effect of each parameter.

In [ ]:
???